In [ ]:
import pandas as pd

**Create ETL folder structure**

In [ ]:
import os
folders = ["raw", "processed", "output"]

for folder in folders:
    os.makedirs(folder, exist_ok=True)

print("Folders created: raw/, processed/, output/")

Folders created: raw/, processed/, output/


**EXTRACT – Load the csv file**

In [16]:
import pandas as pd
df = pd.read_csv("Superstore.csv", encoding="ISO-8859-1")
print(df.head())
print(df.info())


   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156  11-08-2016  11-11-2016    Second Class    CG-12520   
1       2  CA-2016-152156  11-08-2016  11-11-2016    Second Class    CG-12520   
2       3  CA-2016-138688  06-12-2016   6/16/2016    Second Class    DV-13045   
3       4  US-2015-108966  10-11-2015  10/18/2015  Standard Class    SO-20335   
4       5  US-2015-108966  10-11-2015  10/18/2015  Standard Class    SO-20335   

     Customer Name    Segment        Country             City  ...  \
0      Claire Gute   Consumer  United States        Henderson  ...   
1      Claire Gute   Consumer  United States        Henderson  ...   
2  Darrin Van Huff  Corporate  United States      Los Angeles  ...   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   

  Postal Code  Region       Product ID         Category Sub-Category  \
0       42420   Sout

**Save raw data**

In [17]:
df.to_csv("raw/superstore_raw.csv", index=False)

**Remove duplicates**

In [18]:
before = df.shape[0]
df = df.drop_duplicates()
after = df.shape[0]
print("Rows before:", before)
print("Rows after:", after)

Rows before: 9994
Rows after: 9994


**Convert date columns to datetime**

In [20]:
df["Order Date"] = pd.to_datetime(df["Order Date"], format="mixed")
df["Ship Date"] = pd.to_datetime(df["Ship Date"], format="mixed")

In [21]:
print(df[["Order Date", "Ship Date"]].dtypes)

Order Date    datetime64[ns]
Ship Date     datetime64[ns]
dtype: object


**Create DERIVED columns**

In [22]:
df["Profit_Margin"] = (df["Profit"] / df["Sales"]) * 100

In [23]:
df["High_Discount"] = df["Discount"].apply(lambda x: "Yes" if x > 0.3 else "No")

**Save PROCESSED data**

In [24]:
df.to_csv("processed/superstore_processed.csv", index=False)

**Split into multiple tables**

**Customers table**

In [25]:
customers = df[[
    "Customer ID", "Customer Name", "Segment",
    "Country", "City", "State", "Region"
]].drop_duplicates()

customers.to_csv("output/customers.csv", index=False)

**Products table**

In [26]:
products = df[[
    "Product ID", "Product Name", "Category", "Sub-Category"
]].drop_duplicates()

products.to_csv("output/products.csv", index=False)

**Orders table**

In [27]:
orders = df[[
    "Order ID", "Order Date", "Ship Date", "Ship Mode",
    "Customer ID", "Product ID",
    "Sales", "Quantity", "Discount", "Profit",
    "Profit_Margin", "High_Discount"
]]

orders.to_csv("output/orders.csv", index=False)

**Create SQLite database**

In [28]:
import sqlite3
conn = sqlite3.connect("output/superstore.db")

**Load tables into database**

In [29]:
customers.to_sql("customers", conn, if_exists="replace", index=False)
products.to_sql("products", conn, if_exists="replace", index=False)
orders.to_sql("orders", conn, if_exists="replace", index=False)
conn.close()

**Validate row counts**

In [30]:
print("Original rows:", df.shape[0])
print("Customers:", customers.shape[0])
print("Products:", products.shape[0])
print("Orders:", orders.shape[0])

Original rows: 9994
Customers: 4688
Products: 1894
Orders: 9994


**Row count comparison**

In [31]:
print("Original rows:", before)
print("Processed rows:", df.shape[0])
print("Rows removed (duplicates):", before - df.shape[0])

Original rows: 9994
Processed rows: 9994
Rows removed (duplicates): 0


**Check date type difference**

In [32]:
raw_df = pd.read_csv("raw/superstore_raw.csv", encoding="ISO-8859-1")
print(raw_df[["Order Date", "Ship Date"]].dtypes)

Order Date    object
Ship Date     object
dtype: object


In [33]:
print(df[["Order Date", "Ship Date"]].dtypes)

Order Date    datetime64[ns]
Ship Date     datetime64[ns]
dtype: object


**Check newly created columns**

In [34]:
original_cols = set(raw_df.columns)
processed_cols = set(df.columns)
print("New columns added:", processed_cols - original_cols)

New columns added: {'High_Discount', 'Profit_Margin'}


**summary table**

In [35]:
comparison = {
    "Metric": [
        "Total Rows",
        "Total Columns",
        "Duplicate Rows",
        "Date Format"
    ],
    "Raw Data": [
        raw_df.shape[0],
        raw_df.shape[1],
        raw_df.duplicated().sum(),
        "Object"
    ],
    "Processed Data": [
        df.shape[0],
        df.shape[1],
        df.duplicated().sum(),
        "Datetime"
    ]
}
pd.DataFrame(comparison)

,Metric,Raw Data,Processed Data
0,Total Rows,9994,9994
1,Total Columns,21,23
2,Duplicate Rows,0,0
3,Date Format,Object,Datetime


This project demonstrates a mini ETL (Extract, Transform, Load) pipeline using
the Superstore Sales dataset. Raw data was extracted from a CSV file using
Python and pandas. The transformation stage involved removing duplicate records,
converting date columns into datetime format, and creating derived business
columns such as profit margin and high discount flags. The transformed data was
then split into customers, products, and orders tables. Finally, the processed
data was loaded into CSV files and a SQLite database. The pipeline was validated
by comparing row counts and schema changes before and after transformation.
